In [1]:
%use kandy, dataframe

In [2]:
val originalDf = DataFrame.read("reports/jmh/results.csv")
originalDf

Benchmark,Mode,Threads,Samples,Score,Score Error (99.9%),Unit,Param: size
kotlin_benchmarks.array.copy.Identity...,avgt,1,15,9.503013,0.101812,ns/op,100
kotlin_benchmarks.array.copy.Identity...,avgt,1,15,75935.711605,2828.834707,ns/op,1000000
kotlin_benchmarks.array.copy.Identity...,avgt,1,15,76.377792,2.393063,ns/op,100
kotlin_benchmarks.array.copy.Identity...,avgt,1,15,39222433.039542,11875068.776673,ns/op,1000000
kotlin_benchmarks.array.copy.Inline12...,avgt,1,15,9.498100,0.131004,ns/op,100
kotlin_benchmarks.array.copy.Inline12...,avgt,1,15,72904.250853,1612.786780,ns/op,1000000
kotlin_benchmarks.array.copy.Inline12...,avgt,1,15,173.083743,0.857272,ns/op,100
kotlin_benchmarks.array.copy.Inline12...,avgt,1,15,40415357.530449,9078575.540632,ns/op,1000000
kotlin_benchmarks.array.copy.Inline12...,avgt,1,15,9.407102,0.041122,ns/op,100
kotlin_benchmarks.array.copy.Inline12...,avgt,1,15,72006.475273,1307.687902,ns/op,1000000


In [3]:
enum class TestedLanguage { Java, Kotlin }

In [4]:
val kotlinPackage = "kotlin_benchmarks."
val javaPackage = "org.openjdk.bench.valhalla."
val Language by column<TestedLanguage>()

fun <T, C> DataFrame<T>.removeColumnWithSingleValue(c: ColumnReference<C>): DataFrame<T> {
    val values = this[c].distinct().toList()
    return if (values.size <= 1) remove(c) else error("Multiple values for ${c.name()} found: ${values.joinToString()}")
}

val unit = if (originalDf.isEmpty()) "" else originalDf.Unit.distinct().single()

val df = originalDf
    .removeColumnWithSingleValue(originalDf.Mode)
    .removeColumnWithSingleValue(originalDf.Threads)
    .removeColumnWithSingleValue(originalDf.Samples)
    .removeColumnWithSingleValue(originalDf.Unit)
    .add(Language) {
        when {
            Benchmark.startsWith(kotlinPackage) -> TestedLanguage.Kotlin
            Benchmark.startsWith(javaPackage) -> TestedLanguage.Java
            else -> error("Unknown package: $Benchmark")
        }
    }
    .move(Language).toLeft()
    .update { Benchmark }.with {
        when (this[Language]) {
            TestedLanguage.Java -> it.removePrefix(javaPackage)
            TestedLanguage.Kotlin -> it.removePrefix(kotlinPackage)
        }
    }
df

Language,Benchmark,Score,Score Error (99.9%),Param: size
Kotlin,array.copy.Identity32int.ref_arraycopy,9.503013,0.101812,100
Kotlin,array.copy.Identity32int.ref_arraycopy,75935.711605,2828.834707,1000000
Kotlin,array.copy.Identity32int.ref_copy,76.377792,2.393063,100
Kotlin,array.copy.Identity32int.ref_copy,39222433.039542,11875068.776673,1000000
Kotlin,array.copy.Inline128int.Ref_to_Val_ar...,9.498100,0.131004,100
Kotlin,array.copy.Inline128int.Ref_to_Val_ar...,72904.250853,1612.786780,1000000
Kotlin,array.copy.Inline128int.Ref_to_Val_copy,173.083743,0.857272,100
Kotlin,array.copy.Inline128int.Ref_to_Val_copy,40415357.530449,9078575.540632,1000000
Kotlin,array.copy.Inline128int.Val_to_Ref_ar...,9.407102,0.041122,100
Kotlin,array.copy.Inline128int.Val_to_Ref_ar...,72006.475273,1307.687902,1000000


In [5]:
df.Benchmark.distinct().size()

388

In [6]:
df.map { Benchmark to `Param - size` }.distinct().size

680

In [7]:
val groupedDf = df
    .update { Benchmark }
    .with {
        val parameter = `Param - size`
        if (parameter == null) it else "$it (size: $parameter)"
    }
    .groupBy { Benchmark }
groupedDf

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_1426063368"></table>

<p class="dataframe_description">... showing only top 20 of 680 rows</p><p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].pa

In [8]:
groupedDf
    .filter { it.group().rowsCount() != TestedLanguage.entries.size }
    .concat()
    .remove(df.Score, df.`Score Error (99 9%)`, df.`Param - size`)
    //.move { Benchmark }.toLeft()

""


In [9]:
data class CombinedResult(val kotlin: Double, val java: Double)

val KotlinToJava by column<CombinedResult>()

val dataToRender = groupedDf
    .filter { it.group().rowsCount() == TestedLanguage.entries.size }
    .updateGroups { g ->
        g.add(KotlinToJava) {
            val kotlin = g.single { it[g.Language] == TestedLanguage.Kotlin }.Score
            val java = g.single { it[g.Language] == TestedLanguage.Java }.Score
            CombinedResult(kotlin, java)
        }.remove(`Param - size`, Score, `Score Error (99 9%)`, Language).distinct()
    }
    .concat()
dataToRender

Benchmark,KotlinToJava
array.copy.Identity32int.ref_arraycop...,"CombinedResult(kotlin=9.503013, java=..."
array.copy.Identity32int.ref_arraycop...,"CombinedResult(kotlin=75935.711605, j..."
array.copy.Identity32int.ref_copy (si...,"CombinedResult(kotlin=76.377792, java..."
array.copy.Identity32int.ref_copy (si...,CombinedResult(kotlin=3.9222433039542...
array.copy.Inline128int.Ref_to_Val_ar...,"CombinedResult(kotlin=9.4981, java=9...."
array.copy.Inline128int.Ref_to_Val_ar...,"CombinedResult(kotlin=72904.250853, j..."
array.copy.Inline128int.Ref_to_Val_co...,"CombinedResult(kotlin=173.083743, jav..."
array.copy.Inline128int.Ref_to_Val_co...,CombinedResult(kotlin=4.0415357530449...
array.copy.Inline128int.Val_to_Ref_ar...,"CombinedResult(kotlin=9.407102, java=..."
array.copy.Inline128int.Val_to_Ref_ar...,"CombinedResult(kotlin=72006.475273, j..."


In [10]:
fun makePlot(width: Int, height: Int) = dataToRender
    .plot {
        points {
            x(Benchmark)
            y(KotlinToJava.map { it.kotlin / (it.java + it.kotlin) }) {
                scale = continuous(0.0, 1.0)
                size = 2.0
            }
            tooltips(
                KotlinToJava.map { "${it.kotlin} $unit" }.named("Kotlin"),
                KotlinToJava.map { "${it.java} $unit" }.named("Java"),
            )
            color = Color.RED
        }
        hLine {
            yIntercept.constant(0.5)
            color = Color.hex("#756bb1")
            type = LineType.DASHED
            alpha = 0.9
            tooltips(enable = false)
        }
        layout {
            title = "Ratio of Kotlin and Java Valhalla value classes performance"
            size = width to height
            style {
                plotCanvas.title {
                    hJust = 0.5
                }
            }
        }
    }

makePlot(2400, 800).apply {
    save("plot.html", path = "reports/jmh")
    save("plot.svg", path = "reports/jmh")
}
makePlot(800, 800)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="x06etQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Ratio of Kotlin and Java Valhalla value classes performance"
},
"mapping":{
},
"data":{
"Java":["11.012168 ns/op","80345.729635 ns/op","76.275049 ns/op","3.2344425788752E7 ns/op","9.568516 ns/op","78808.214197 ns/op","178.592043 ns/op","2.7545211396094E7 ns/op","16.946826 ns/op","196560.60965 ns/op","178.935163 ns/op","2.6199026865901E7 ns/op","9.634774 ns/op","81077.433683 ns/op","177.676875 ns/op","2.912059872752E7 ns/op","9.976317 ns/op","81634.1269 ns/op","77.69404 ns/op","2.7103847243326E7 ns/op","14.389643 ns/op","192329.453177 ns/op","78.060437 ns/op","4.1580340401004E7 ns/op","9.9288 ns/op","81254.769488 ns/op","78.20296 ns/op","2.3110511037593E7 ns/op","9.645978 ns/op","85270.484856 ns/op","191.808635 ns/op","2.7014355743596E7 ns/op","15.978698 ns/op","141046.32164 ns/op","179.19803 ns/op","2.3443055461045E7 ns/op","9.635434 ns/op","80216.448186 ns/op","179.14044 ns/op","2.5225103420707E7 ns/op","9.694026 ns/op","89421.621525 ns/op","197.988157 ns/op","3.5098664931361E7 ns/op","13.152362 ns/op","82401.624022 ns/op","179.00298 ns/op","2.3804121076667E7 ns/op","9.64246 ns/op","86816.54798 ns/op","195.584235 ns/op","2.5193232044058E7 ns/op","21.244602 ns/op","81685.821755 ns/op","77.88505 ns/op","2.1573679389831E7 ns/op","10.011787 ns/op","82669.286693 ns/op","77.90784 ns/op","2.2360645236961E7 ns/op","10.033584 ns/op","80895.108665 ns/op","77.982658 ns/op","2.2963178917891E7 ns/op","28.810631 ns/op","411614.547753 ns/op","21.423691 ns/op","560731.948432 ns/op","7.723096 ns/op","75256.949901 ns/op","9.372902 ns/op","90214.735445 ns/op","15.434207 ns/op","163530.617977 ns/op","19.742666 ns/op","201563.38381 ns/op","15.493624 ns/op","164919.750256 ns/op","12.044675 ns/op","190987.429094 ns/op","15.473341 ns/op","163992.359429 ns/op","10.998395 ns/op","162760.304341 ns/op","88.702368 ns/op","2.3607423303889E7 ns/op","180.217357 ns/op","1821812.335527 ns/op","89.875458 ns/op","2.2216161864887E7 ns/op","85.206681 ns/op","2.3031809873481E7 ns/op","71.915778 ns/op","2.3581093317739E7 ns/op","71.455614 ns/op","2.2927085674615E7 ns/op","73.514145 ns/op","2.4157468966726E7 ns/op","73.981081 ns/op","2.3654239961296E7 ns/op","87.485629 ns/op","2.2976931335487E7 ns/op","180.524399 ns/op","1899817.978749 ns/op","87.656638 ns/op","2.3216890428087E7 ns/op","87.440433 ns/op","2.3403706019706E7 ns/op","71.907327 ns/op","2.4385855505288E7 ns/op","71.632814 ns/op","2.4555263174605E7 ns/op","73.455119 ns/op","2.2708283047748E7 ns/op","73.904151 ns/op","2.3912586290501E7 ns/op","136.631342 ns/op","2.4608881729818E7 ns/op","348.416238 ns/op","3569865.041505 ns/op","132.752252 ns/op","2.376047118464E7 ns/op","341.77004 ns/op","3621805.867478 ns/op","132.37322 ns/op","2.3980303485004E7 ns/op","250.138562 ns/op","2952039.214558 ns/op","132.800536 ns/op","2.4243159626998E7 ns/op","254.708429 ns/op","2947548.554904 ns/op","117.062156 ns/op","2.3930259899324E7 ns/op","117.716731 ns/op","2.4562582600222E7 ns/op","131.331446 ns/op","2.6783821710726E7 ns/op","133.629996 ns/op","2.6439037722012E7 ns/op","117.149463 ns/op","2.4716807779041E7 ns/op","117.208133 ns/op","2.3974881221295E7 ns/op","130.583826 ns/op","2.5885807794795E7 ns/op","134.579471 ns/op","2.6869324940528E7 ns/op","117.275159 ns/op","2.4405165686815E7 ns/op","118.119491 ns/op","2.5485272133453E7 ns/op","129.47126 ns/op","2.9939286788996E7 ns/op","133.56232 ns/op","4.9757325362035E7 ns/op","117.47711 ns/op","4.9016979846753E7 ns/op","118.389113 ns/op","4.7961783762826E7 ns/op","127.863487 ns/op","5.6548657096959E7 ns/op","130.313398 ns/op","6.0839469184139E7 ns/op","67.981781 ns/op","4.85003055